In [347]:
import pandas as pd
import json
import numpy as np
from pandas import json_normalize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

In [348]:


def fake_matrix(nb_products = 6,
                nb_cust = 5,
                share_empty = 0.7):

    name_products = ["P"+str(x+1) for x in range(nb_products)]
    name_cust = ["cust_"+str(x+1) for x in range(nb_cust)]

    rand = np.random.rand(nb_cust,nb_products)
    empty = np.random.rand(nb_cust,nb_products)>share_empty

    fake_matrix = pd.DataFrame(rand*empty,
                               index=name_cust,
                               columns=name_products
                              )


    return fake_matrix

# Item-based recommendation

In [349]:
#Generate DtaFrame
fake=fake_matrix(1000, 5, 0.7)
fake.info()
fake.head()

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, cust_1 to cust_5
Columns: 1000 entries, P1 to P1000
dtypes: float64(1000)
memory usage: 39.1+ KB


,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,...,P991,P992,P993,P994,P995,P996,P997,P998,P999,P1000
cust_1,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.492871,0.000000,0.0,...,0.730142,0.0,0.00000,0.0,0.160712,0.000000,0.824349,0.000000,0.888260,0.000000
cust_2,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.736519,0.000000,0.000000,0.000000,0.000000
cust_3,0.000000,0.000000,0.0,0.0,0.0,0.569211,0.0,0.708124,0.000000,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
cust_4,0.281566,0.000000,0.0,0.0,0.0,0.000000,0.0,0.817431,0.000000,0.0,...,0.000000,0.0,0.20397,0.0,0.000000,0.000000,0.000000,0.759802,0.120324,0.000000
cust_5,0.000000,0.864026,0.0,0.0,0.0,0.000000,0.0,0.000000,0.440273,0.0,...,0.104698,0.0,0.32419,0.0,0.000000,0.000000,0.541161,0.310371,0.316506,0.736489


In [350]:
#Transposition
fake_t=fake.transpose()
fake_t

,cust_1,cust_2,cust_3,cust_4,cust_5
P1,0.000000,0.000000,0.0,0.281566,0.000000
P2,0.000000,0.000000,0.0,0.000000,0.864026
P3,0.000000,0.000000,0.0,0.000000,0.000000
P4,0.000000,0.000000,0.0,0.000000,0.000000
P5,0.000000,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...
P996,0.000000,0.736519,0.0,0.000000,0.000000
P997,0.824349,0.000000,0.0,0.000000,0.541161
P998,0.000000,0.000000,0.0,0.759802,0.310371
P999,0.888260,0.000000,0.0,0.120324,0.316506


In [351]:
#Normalisation
fake_t_norm = fake_t.apply(lambda x: x - np.nanmean(x), axis=0)
fake_t_norm.head(20)

,cust_1,cust_2,cust_3,cust_4,cust_5
P1,-0.163763,-0.143250,-0.162591,0.127214,-0.153379
P2,-0.163763,-0.143250,-0.162591,-0.154352,0.710648
P3,-0.163763,-0.143250,-0.162591,-0.154352,-0.153379
P4,-0.163763,-0.143250,-0.162591,-0.154352,-0.153379
P5,-0.163763,-0.143250,-0.162591,-0.154352,-0.153379
P6,-0.163763,-0.143250,0.406620,-0.154352,-0.153379
P7,-0.163763,-0.143250,-0.162591,-0.154352,-0.153379
P8,0.329108,-0.143250,0.545533,0.663079,-0.153379
P9,-0.163763,-0.143250,-0.162591,-0.154352,0.286894
P10,-0.163763,-0.143250,-0.162591,-0.154352,-0.153379


In [352]:
#Calucation similarity (for the sake of example)
item_similarity_cosine = cosine_similarity(fake_t_norm)
item_similarity_cosine

array([[ 1.        , -0.20977818,  0.66242841, ...,  0.55163037,
        -0.38733087, -0.15839119],
       [-0.20977818,  1.        , -0.04219117, ...,  0.17358687,
         0.08232854,  0.99698555],
       [ 0.66242841, -0.04219117,  1.        , ..., -0.18437648,
        -0.33936307,  0.03545439],
       ...,
       [ 0.55163037,  0.17358687, -0.18437648, ...,  1.        ,
        -0.12628328,  0.15931429],
       [-0.38733087,  0.08232854, -0.33936307, ..., -0.12628328,
         1.        ,  0.05599632],
       [-0.15839119,  0.99698555,  0.03545439, ...,  0.15931429,
         0.05599632,  1.        ]])

In [353]:
#Calucation similarity matrix among items
item_similarity = pd.DataFrame(cosine_similarity(fake_t_norm, fake_t_norm), index=fake_t_norm.index, columns=fake_t_norm.index)
item_similarity.head()

,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,...,P991,P992,P993,P994,P995,P996,P997,P998,P999,P1000
P1,1.000000,-0.209778,0.662428,0.662428,0.662428,-0.086647,0.662428,-0.044677,0.070936,0.662428,...,-0.274147,0.662428,0.492697,0.662428,0.496301,-0.122856,-0.513743,0.551630,-0.387331,-0.158391
P2,-0.209778,1.000000,-0.042191,-0.042191,-0.042191,-0.262592,-0.042191,-0.455368,0.915613,-0.042191,...,-0.116096,-0.042191,0.743941,-0.042191,-0.158209,-0.223856,0.378375,0.173587,0.082329,0.996986
P3,0.662428,-0.042191,1.000000,1.000000,1.000000,0.161017,1.000000,-0.611997,0.363071,1.000000,...,-0.066400,1.000000,0.353264,1.000000,0.887008,0.066705,-0.343276,-0.184376,-0.339363,0.035454
P4,0.662428,-0.042191,1.000000,1.000000,1.000000,0.161017,1.000000,-0.611997,0.363071,1.000000,...,-0.066400,1.000000,0.353264,1.000000,0.887008,0.066705,-0.343276,-0.184376,-0.339363,0.035454
P5,0.662428,-0.042191,1.000000,1.000000,1.000000,0.161017,1.000000,-0.611997,0.363071,1.000000,...,-0.066400,1.000000,0.353264,1.000000,0.887008,0.066705,-0.343276,-0.184376,-0.339363,0.035454


In [354]:
cust="cust_2"
item="P2"
n=5

In [355]:
#Items already rated by customer
rated_items = pd.DataFrame(fake_t_norm[cust].dropna(axis=0, how='all')\
                          .sort_values(ascending=False))\
                          .reset_index()\
                          .rename(columns={1:'rating'})
rated_items.head()

,index,cust_2
0,P53,0.856744
1,P331,0.852829
2,P469,0.849646
3,P233,0.846399
4,P496,0.837408


In [356]:
# Similarity score of a spedcific Item  with all the other 
picked_item_similarity_score = item_similarity[[item]].reset_index().rename(columns={item:'similarity_score'})

# Rank the similarities between the items rated by customer and picked item.

picked_cust_rated_similarity = pd.merge(left=rated_items, 
                                            right=picked_item_similarity_score, 
                                            on='index', 
                                            how='inner')\
                                     .sort_values('similarity_score', ascending=False)[:n]

In [357]:
picked_cust_rated_similarity

,index,cust_2,similarity_score
811,P2,-0.14325,1.000000
367,P636,-0.14325,0.999997
885,P531,-0.14325,0.999989
748,P117,-0.14325,0.999954
382,P984,-0.14325,0.999925


In [358]:
# Calculate the predicted rating using weighted average of similarity scores and the ratings from picked customer

predicted_score = round(np.average(picked_cust_rated_similarity['similarity_score'], 
                                    weights=picked_cust_rated_similarity['similarity_score']), 6)
print(f'The predicted rating for {item} by user {cust} is {predicted_score}' )



The predicted rating for P2 by user cust_2 is 0.999973


## Create recommender with N items

In [359]:
#Item based recommendation

import operator
def item_based_rec(picked_userid=cust, number_of_similar_items=10, number_of_recommendations =5):

# Items that the target customer has not rated

    picked_cust_unrate = pd.DataFrame(fake_t_norm[cust]==0).reset_index()

    picked_cust_unrate = picked_cust_unrate["index"].values.tolist()

    # Items that the customer rated
    picked_cust_rate = pd.DataFrame(fake_t_norm[cust].dropna(axis=0, how='all')\
                            .sort_values(ascending=False))\
                            .reset_index()\
                            .rename(columns={1:'rating'})
   
    # Dictionary to save the unrated items and predicted rating pair
    rating_prediction ={}  
    
    # Loop through unrated items          
    

    for picked_item in picked_cust_unrate: 
        
         # Calculate the similarity score of the picked item with other items 
        picked_item_similarity_score = item_similarity[[picked_item]].reset_index().rename(columns={picked_item:'similarity_score'})
    
    # Rank the similarities between the picked item customer rated item and the picked unrated item.
    
        picked_customer_rated_similarity = pd.merge(left=rated_items, 
                                                right=picked_item_similarity_score, 
                                                on='index', 
                                                how='inner')\
                                        .sort_values('similarity_score', ascending=False)[:number_of_similar_items]
    
    # Calculate the predicted rating using weighted average of similarity scores and the ratings from customer
    
        predicted_rating = round(np.average(picked_customer_rated_similarity['similarity_score'], 
                                        weights=picked_customer_rated_similarity['similarity_score']), 6)
   
    # Save the predicted rating in the dictionary
        rating_prediction[picked_item] = predicted_rating
    
    # Return the top recommended movies
    return sorted(rating_prediction.items(), key=operator.itemgetter(1), reverse=True)[:number_of_recommendations]
    


In [363]:
# Get recommendations

recommended_movie = item_based_rec(cust, 500, 10)
recommended_movie


[('P269', 1.054578),
 ('P38', 1.022916),
 ('P331', 1.01448),
 ('P406', 0.997317),
 ('P740', 0.994094),
 ('P239', 0.990713),
 ('P304', 0.986649),
 ('P771', 0.986203),
 ('P890', 0.982208),
 ('P537', 0.98037)]

# With KNNMeans

In [ ]:
!pip install scikit-surprise

In [405]:
from surprise import Dataset
from surprise import Reader
from surprise import KNNWithMeans
from surprise.model_selection import cross_validate, train_test_split
from surprise.accuracy import rmse, mae
from surprise.model_selection import GridSearchCV

In [369]:
#orders_scoring = pd.read_json("../raw_data/order_scoring.json",lines=True)
orders_scoring = pd.read_json("order_scoring.json",lines=True)

In [378]:
orders_scoring.head(20)

,customer_id,LO,sku
0,h6IvomPZxOTKuKGuvS8b3nZrzVk2,1,1-CTR-103
1,h6IvomPZxOTKuKGuvS8b3nZrzVk2,1,2-END-103
2,h6IvomPZxOTKuKGuvS8b3nZrzVk2,1,1-LIM-136
3,h6IvomPZxOTKuKGuvS8b3nZrzVk2,1,1-ECM-114
4,h6IvomPZxOTKuKGuvS8b3nZrzVk2,1,6-VDG-101
5,h6IvomPZxOTKuKGuvS8b3nZrzVk2,1,6-LSD-107
6,h6IvomPZxOTKuKGuvS8b3nZrzVk2,1,1-DEM-102
7,h6IvomPZxOTKuKGuvS8b3nZrzVk2,1,1-HRB-109
8,h6IvomPZxOTKuKGuvS8b3nZrzVk2,1,3-GRZ-103
9,h6IvomPZxOTKuKGuvS8b3nZrzVk2,1,6-LSD-109


In [396]:
reader = Reader(rating_scale=(-0.5,1))
data = Dataset.load_from_df(orders_scoring[["customer_id", "sku", "LO"]], reader)

In [390]:
type(data)

surprise.dataset.DatasetAutoFolds

In [397]:
sim_options = {
    "name": "cosine",
    "user_based": False,  # Compute  similarities between items
}
model = KNNWithMeans(sim_options=sim_options)

In [398]:
trainset, testset = train_test_split(data, test_size=0.25)

model.fit(trainset)



Computing the cosine similarity matrix...
Done computing similarity matrix.


In [400]:
predictions = model.test(testset)

In [401]:
rmse(predictions)

RMSE: 1.0208


1.0207525579121997

In [402]:
mae(predictions)

MAE:  0.8692


0.8691635353931925

In [403]:
cross_validate(model, data, measures=['RMSE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0205  1.0217  1.0206  1.0203  1.0211  1.0208  0.0005  
Fit time          22.35   22.47   41.05   24.51   32.77   28.63   7.29    
Test time         133.27  174.07  160.95  196.45  152.29  163.41  21.18   


{'test_rmse': array([1.02051067, 1.02166577, 1.0206498 , 1.02026099, 1.02108035]),
 'fit_time': (22.34518814086914,
  22.467735052108765,
  41.05263590812683,
  24.514350175857544,
  32.77282214164734),
 'test_time': (133.27049779891968,
  174.0743706226349,
  160.94648599624634,
  196.45489501953125,
  152.2921531200409)}

## Tunning model

In [ ]:
sim_opt = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}


param_grid = {"sim_options": sim_opt}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


In [1]:
!pwd

/Users/gulnaramusina/code/PaulChabert/raw_data
